In [147]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive
from astropy.coordinates import SkyCoord

Define observing constraints: 

In [148]:
# Equilibrium temperatures
teq_min = 0 * u.K
teq_max = 1000 * u.K

rad_min = 0 * u.R_earth
rad_max = 10 * u.R_earth

gaia_mag_limit = 15

mean_molecular_mass = 6 * u.u
no_mass_standard_density = 2.9 * u.g / u.cm**3


Download NASA Exoplanet Archive table from the web via astroquery:

In [135]:
table = NasaExoplanetArchive.get_confirmed_planets_table(all_columns=True)
transiting = table[(table['pl_tranflag'].data).astype(bool)]

Compute equilibrium temperatures for zero albedo: 

In [136]:
rp = transiting['pl_radj']
equilibrium_temperature = (transiting['st_teff'] * np.sqrt(transiting['st_rad'] / 2 / transiting['pl_orbsmax'])).decompose()
g = transiting['gaia_gmag'].quantity

transiting['teq'] = equilibrium_temperature

gp = const.G * 10*u.earthMass / rp**2
transiting['H'] = ((const.k_B * equilibrium_temperature) / (mean_molecular_mass * gp)).to(u.km)
transiting['signal'] = (2*rp*transiting['H']/transiting['st_rad']**2).decompose()

[47.59955554 54.15771652 17.51692725 ... 63.29608944 16.71534461
 21.60307806] m / s2


In [137]:
temp_constraints = (equilibrium_temperature < teq_max) & (equilibrium_temperature > teq_min)
rad_constraints = (rp < rad_max) & (rp > rad_min)
gmag_constaints = (g < gaia_mag_limit)

targets = transiting[rad_constraints & temp_constraints & gmag_constaints]

In [138]:
targets.sort('st_j')

In [139]:
targets['r_earth'] = targets['pl_radj'].to(u.R_earth)
print(targets['pl_trandur'])

pl_trandur
----------
 0.0691667
 0.1047917
 0.1345833
        --
        --
        --
        --
    0.0574
   0.06375
        --
       ...
    0.2067
   0.55875
    0.1302
    0.1568
    0.2273
    0.2673
    0.2513
    0.3125
     0.275
    0.2875
    0.0704
Length = 602 rows


In [140]:
targets['teq']=np.round(targets['teq'],0)
targets['r_earth']=np.round(targets['r_earth'],2)
targets['H']=np.round(targets['H'],0)
targets['pl_orbper']=np.round(targets['pl_orbper'],2)
targets['dec']=np.round(targets['dec'],1)
targets['duration']=np.round(targets['pl_trandur']*24.0,1)
targets['signal']=np.round(1e6*targets['signal'],2)
targets[['pl_name', 'st_j', 'teq', 'r_earth', 'pl_bmasse','pl_orbper','H','signal','duration','dec','st_spstr']].show_in_notebook()

idx,pl_name,st_j,teq,r_earth,pl_bmasse,pl_orbper,H,signal,duration,dec,st_spstr
,,,K,earthRad,,d,km,,,deg,
0,HD 219134 c,3.981,783.0,1.51,4.36,6.76,25.0,1.66,1.7,57.2,K3 V
1,HD 21749 c,6.081,705.0,0.9,3.7,7.79,8.0,0.4,2.5,-63.5,K4.5
2,GJ 143 b,6.081,425.0,2.61,22.7,35.61,41.0,5.92,3.2,-63.5,K4.5
3,Kepler-444 c,7.244,954.0,0.49,--,4.55,3.0,0.08,--,41.6,K
4,Kepler-444 d,7.244,860.0,0.53,--,6.19,3.0,0.08,--,41.6,K
5,Kepler-444 e,7.244,799.0,0.55,--,7.74,3.0,0.09,--,41.6,K
6,Kepler-444 f,7.244,740.0,0.74,--,9.74,6.0,0.2,--,41.6,K
7,LTT 1445 A b,7.294,436.0,1.38,2.2,5.36,12.0,5.44,1.4,-16.6,M3
8,GJ 357 b,7.337,527.0,1.22,1.84,3.93,11.0,3.1,1.5,-21.7,M2.5 V


In [146]:
table[['pl_name','pl_orbper']].show_in_notebook()

idx,pl_name,pl_orbper
,,d
0,Kepler-128 b,15.09
1,Kepler-128 c,22.804
2,Kepler-129 b,15.79186
3,Kepler-129 c,82.20017
4,Kepler-130 b,8.457458
5,Kepler-130 c,27.50868
6,Kepler-130 d,87.517905
7,Kepler-131 b,16.092
8,Kepler-131 c,25.5169
